## Загрузка библиотек, подключение Гугл-Диска

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier

In [3]:
# Подключаем Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Предобработка исходной таблицы с сохранением результатов на диск

In [4]:
# Загрузка базы данных
df = pd.read_excel('/content/drive/MyDrive/Стажировка/Рак_легких/отправка_комбинации поддержка после ПХТ 3 стадия.xlsx')


In [5]:
# Исходный размер таблицы данных и ее вид
print(df.shape)
print(df.columns.tolist())
#df.head()

(248832, 21)
['id_ответа', 'Раса', 'Пол', 'Возраст', 'Статус курения', 'ECOG', 'Есть опухолевая нагрузка? (симптомная опухоль)', 'Ко-мутации KRAS', 'Ко-мутации p53.', 'Ко-мутации STK11', 'Ко-мутации KEAP1', 'Срок от окончания ХЛТ', 'Молекулярный статус (только для неплоскоклеточного рака)', 'PD-L1 статус', 'Предпочтение пациента по ответу на терапию', 'Ответ эксперта (Лактионов)', 'Поставьте галочку, если уверены на 100%', 'Если 75%', 'Если 50%', 'Альтернатива, если 50% - обязательно', 'Комментарий']


In [6]:
# Функция очистки таблицы
# из всех ячеек, кроме типа bool, удалим служебные символы и приведем к нижненму регистру
def clean_and_lower(x):
    if isinstance(x, bool):      # Проверяем, является ли значение ячейки типом bool
        return x                 # Возвращаем значение ячейки без изменений
    else:
        if isinstance(x, str):                                         # Проверяем, является ли значение ячейки строкой
            x = ''.join(e for e in x if e.isalnum() or e.isspace())    # Удаляем служебные символы из строки
            x = x.lower()                                              # Приводим символы к нижнему регистру
        return x

In [7]:
# Обработка таблицы
cleaned_df = df.applymap(clean_and_lower)

In [8]:
# Оставим только нужные нам для работы столбцы
selected_columns = ['Срок от окончания ХЛТ', 'Молекулярный статус (только для неплоскоклеточного рака)', 'PD-L1 статус', 'Предпочтение пациента по ответу на терапию',
                    'Ответ эксперта (Лактионов)', 'Поставьте галочку, если уверены на 100%', 'Если 75%', 'Если 50%', 'Альтернатива, если 50% - обязательно', 'Комментарий']
new_df = cleaned_df[selected_columns.copy()]
print(new_df.shape)

(248832, 10)


In [9]:
# Создадим словарь категориальных признаков "new_df"
dicts = {}
for i in new_df.columns.values.tolist():
    if new_df[i].dtypes == object:
        c = 0
        dl = {}
        for j in new_df[i].unique():
            c += 1
            dl.update({j:c})
        dicts.update({i:dl})

In [10]:
dicts

{'Срок от окончания ХЛТ': {'до 42 дней': 1,
  'от 43 до 60 дней': 2,
  'более 61 дня': 3},
 'Молекулярный статус (только для неплоскоклеточного рака)': {' нет мутаций': 1,
  ' не исследовались': 2,
  ' egfr редкий вариант': 3,
  ' egfr ex19': 4,
  ' egfr ex21': 5,
  ' alk позитивный': 6},
 'PD-L1 статус': {' не исследовался': 1, ' менее 1': 2, ' более 1': 3},
 'Предпочтение пациента по ответу на терапию': {' результативность лечения': 1,
  ' сохранение качества жизни': 2},
 'Ответ эксперта (Лактионов)': {'дурвалумаб': 1,
  'наблюдение': 2,
  'осимертиниб': 3,
  'алектиниб': 4},
 'Альтернатива, если 50% - обязательно': {0: 1,
  'дурвалумаб': 2,
  'наблюдение': 3,
  'осимертиниб': 4,
  'алектиниб': 5},
 'Комментарий': {0: 1,
  'нет доказательной базы для назначения после перерыва 61 дня после хлт': 2,
  'возможно есть мутации и эффективность дурвалумаба будет низкой': 3,
  'возможна низкая эффективность дурвалумаба осимертиниб без доказательной базы': 4,
  'возможна низкая эффективность 

In [11]:
# Соберем таблицу категориальных признаков на базе "new_df" и "dicts"
df_fl = new_df.copy()
for i in dicts.keys():
    for j in range(new_df[i].shape[0]):
        df_fl[i][j] = float(dicts.get(i).get(new_df[i][j]))

In [12]:
display(new_df.head())
display(df_fl.head())

,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Поставьте галочку, если уверены на 100%",Если 75%,Если 50%,"Альтернатива, если 50% - обязательно",Комментарий
0,до 42 дней,нет мутаций,не исследовался,результативность лечения,дурвалумаб,True,False,False,0,0
1,от 43 до 60 дней,нет мутаций,не исследовался,результативность лечения,дурвалумаб,True,False,False,0,0
2,более 61 дня,нет мутаций,не исследовался,результативность лечения,наблюдение,False,True,False,дурвалумаб,нет доказательной базы для назначения после пе...
3,до 42 дней,не исследовались,не исследовался,результативность лечения,дурвалумаб,False,True,False,0,возможно есть мутации и эффективность дурвалум...
4,от 43 до 60 дней,не исследовались,не исследовался,результативность лечения,дурвалумаб,False,True,False,0,возможно есть мутации и эффективность дурвалум...


,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Поставьте галочку, если уверены на 100%",Если 75%,Если 50%,"Альтернатива, если 50% - обязательно",Комментарий
0,1.0,1.0,1.0,1.0,1.0,True,False,False,1.0,1.0
1,2.0,1.0,1.0,1.0,1.0,True,False,False,1.0,1.0
2,3.0,1.0,1.0,1.0,2.0,False,True,False,2.0,2.0
3,1.0,2.0,1.0,1.0,1.0,False,True,False,1.0,3.0
4,2.0,2.0,1.0,1.0,1.0,False,True,False,1.0,3.0


In [13]:
# Соберем решения эксперта в один столбец.
# Для этого создадим сначала функцию анализа решений эксперта

def analyze_row(row):
    if row["Поставьте галочку, если уверены на 100%"] == True:
        return 1
    elif row["Если 75%"] == True:
        return 2
    elif row["Если 50%"] == True:
        return 3
    else:
        count = sum(row[columns_to_analyze])
        if count > 1:
            return 0
        else:
            return None

# Применяем функцию analyze_row к каждой строке таблицы и сохраняем результат в столбец "Решение"
df_fl["Решение"] = df_fl.apply(analyze_row, axis=1)

# Оценим балансировку данных:
df_fl['Решение'].value_counts()

1    89856
2    89856
3    69120
Name: Решение, dtype: int64

In [14]:
# Сохраним на диск:
new_df.to_csv('/content/drive/MyDrive/Стажировка/Рак_легких/Raw_data.csv')
df_fl.to_csv('/content/drive/MyDrive/Стажировка/Рак_легких/Number_data.csv')

## Загузка сохраненных данных с диска и подготовка датасета для НС

In [15]:
df_fl = pd.read_csv('/content/drive/MyDrive/Стажировка/Рак_легких/Number_data.csv', index_col=None)
df_fl.head()

,Unnamed: 0,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Поставьте галочку, если уверены на 100%",Если 75%,Если 50%,"Альтернатива, если 50% - обязательно",Комментарий,Решение
0,0,1.0,1.0,1.0,1.0,1.0,True,False,False,1.0,1.0,1
1,1,2.0,1.0,1.0,1.0,1.0,True,False,False,1.0,1.0,1
2,2,3.0,1.0,1.0,1.0,2.0,False,True,False,2.0,2.0,2
3,3,1.0,2.0,1.0,1.0,1.0,False,True,False,1.0,3.0,2
4,4,2.0,2.0,1.0,1.0,1.0,False,True,False,1.0,3.0,2


In [16]:
df_fl = df_fl.drop(columns = 'Unnamed: 0')
df_fl.nunique()

Срок от окончания ХЛТ                                        3
Молекулярный статус (только для неплоскоклеточного рака)     6
PD-L1 статус                                                 3
Предпочтение пациента по ответу на терапию                   2
Ответ эксперта (Лактионов)                                   4
Поставьте галочку, если уверены на 100%                      2
Если 75%                                                     2
Если 50%                                                     2
Альтернатива, если 50% - обязательно                         5
Комментарий                                                 11
Решение                                                      3
dtype: int64

In [17]:
# Соберем таблицу для обучения сети
df_fin = df_fl.drop(columns = ['Поставьте галочку, если уверены на 100%', 'Если 75%', 'Если 50%', 'Альтернатива, если 50% - обязательно'])

In [18]:
# Нормализуем данные в таблице (кроме ячеек 'Решение')
for i in df_fin.columns.values.tolist():
   if i != 'Решение':
        df_fin[i] = df_fin[i]/df_fin[i].max()

In [19]:
df_fin.head(10)

,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),Комментарий,Решение
0,0.333333,0.166667,0.333333,0.5,0.25,0.090909,1
1,0.666667,0.166667,0.333333,0.5,0.25,0.090909,1
2,1.000000,0.166667,0.333333,0.5,0.50,0.181818,2
3,0.333333,0.333333,0.333333,0.5,0.25,0.272727,2
4,0.666667,0.333333,0.333333,0.5,0.25,0.272727,2
5,1.000000,0.333333,0.333333,0.5,0.50,0.181818,2
6,0.333333,0.500000,0.333333,0.5,0.50,0.363636,1
7,0.666667,0.500000,0.333333,0.5,0.50,0.363636,1
8,1.000000,0.500000,0.333333,0.5,0.50,0.090909,1
9,0.333333,0.666667,0.333333,0.5,0.75,0.363636,2


In [67]:
# Сформируем массив данных и их значений
x=df_fin.drop(columns=['Решение','Ответ эксперта (Лактионов)','Комментарий']).reset_index(drop=True)
y=df_fin['Решение']


In [68]:
x = x.to_numpy()
y = y.to_numpy()

In [69]:
x

array([[0.33333333, 0.16666667, 0.33333333, 0.5       ],
       [0.66666667, 0.16666667, 0.33333333, 0.5       ],
       [1.        , 0.16666667, 0.33333333, 0.5       ],
       ...,
       [0.33333333, 1.        , 1.        , 1.        ],
       [0.66666667, 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        ]])

In [70]:
y

array([1, 1, 2, ..., 3, 3, 1])

In [71]:
# Разобъем массив на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True)

In [72]:
# Вывод размеров обучающей и проверочной выборок
print(f'Структура и размер массива обучающей выборки: {x_train.shape}')
print(f'Структура и размер массива проверочной выборки: {x_test.shape}')

Структура и размер массива обучающей выборки: (199065, 4)
Структура и размер массива проверочной выборки: (49767, 4)


## Визуализация

In [ ]:
# Получение списка значений разницы между y_test и pred
def test_pred_diff_plot(test, pred, method):
    l_dl = []
    print(method)
    for i in range(0, len(test)):
#       print(test[i], pred[i], sep=' ')
        dif = (abs(test[i]-pred[i])/test[i])*100
        if dif != float("inf"):
           l_dl.append(dif)
    return l_dl

In [ ]:
def ml_method_est(method_list,  x_train, x_test, y_train, y_test, plot=False):

    y_train=np.reshape(y_train.values, (-1,1))
    y_test=np.reshape(y_test.values, (-1,1))

    methods = []
    MAE_list = []
    R_2_list = []
    MSE_list = []
    diff_list = []
    valscore_list = []
    plots = []
    for i in method_list:
        model = i.fit(x_train, y_train)
        p1 = model.predict(x_test)

        mae = mean_absolute_error(y_test, p1)
        r2 = r2_score(y_test, p1)
        mse = mean_squared_error(y_test, p1)

        fold = KFold(n_splits = 6, shuffle = True, random_state = 5)
        scores = cross_val_score(estimator = model, X = x_train, y = y_train, cv=fold)

        methods.append(str(i))
        MAE_list.append(mae)
        R_2_list.append(r2)
        MSE_list.append(mse)
        valscore_list.append(scores.mean())

        diff = test_pred_diff_plot(y_test, p1, i)
        plots.append(diff)
        diff_list.append(sum(diff)/len(y_test))

    method_col=pd.Series(methods)
    mae_col = pd.Series(MAE_list)
    r_2_col = pd.Series(R_2_list)
    mse_col = pd.Series(MSE_list)
    valscore_col = pd.Series(valscore_list)
    est_df = pd.DataFrame({'method':method_col, 'mae':mae_col, 'r_2':r_2_col, 'mse':mse_col, 'mean_diff(%)':diff_list, 'mean_val_score':valscore_col})

    count=0
    for k in plots:
        plt.plot(k, label=str(count))
        count += 1
    plt.legend()
    plt.title('Величина различий реальных и предсказанных значений в % в зависимости от методики')
    plt.show()


    return est_df

## Предобученные алгоритмы ML


In [62]:
from lightgbm import LGBMClassifier

In [73]:
lgbm_tuned = LGBMClassifier(boosting_type = 'gbdt'
                            ,force_col_wise=True) # LightGBM Classifier with optimum paramteres
lgbm_tuned.fit(x_train, y_train)

[LightGBM] [Info] Total Bins 18
[LightGBM] [Info] Number of data points in the train set: 199065, number of used features: 4
[LightGBM] [Info] Start training from score -1.016882
[LightGBM] [Info] Start training from score -1.018661
[LightGBM] [Info] Start training from score -1.283013
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(force_col_wise=True)

In [97]:
expected_y  = y_test
predicted_y = lgbm_tuned.predict(x_test)

In [98]:
from sklearn import metrics
print(metrics.classification_report(expected_y, predicted_y))
print(metrics.confusion_matrix(expected_y, predicted_y))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     17850
           2       1.00      1.00      1.00     17978
           3       1.00      1.00      1.00     13939

    accuracy                           1.00     49767
   macro avg       1.00      1.00      1.00     49767
weighted avg       1.00      1.00      1.00     49767

[[17850     0     0]
 [    0 17978     0]
 [    0     0 13939]]


In [105]:
import joblib
# save model
joblib.dump(lgbm_tuned, '/content/drive/MyDrive/Стажировка/Рак_легких/Модель/lgb.pkl')
# load model
#gbm_pickle = joblib.load('/content/drive/MyDrive/Стажировка/Рак_легких/Модель/lgb.pkl')

['/content/drive/MyDrive/Стажировка/Рак_легких/Модель/lgb.pkl']